In [16]:
import pandas as pd
from functions_load_and_transform import player_list, schedule


In [19]:
player_list

['Alex',
 'Alina',
 'Evelyn',
 'Christopher',
 'Ludwig',
 'Manu',
 'Natalie',
 'Nikolai',
 'Sebastian',
 'Vero',
 'Viki',
 'Wolfgang']

In [22]:
scoringDF = pd.read_csv("data/scoring.csv")
scoringDF = scoringDF.drop(player_list, axis=1)
for player in player_list:
    scoringDF = scoringDF.rename(columns={f"score_{player}":player})
scoringDF

,Winner,Week,Game Nr.,Alex,Alina,Evelyn,Christopher,Ludwig,Manu,Natalie,Nikolai,Sebastian,Vero,Viki,Wolfgang
0,Kansas City Chiefs,1,1,1,0,1,0,0,0,0,0,0,0,0,0
1,Philadelphia Eagles,1,2,1,0,1,0,0,0,0,0,0,0,0,0
2,Atlanta Falcons,1,3,1,0,1,0,0,0,0,0,0,0,0,0
3,Draw,1,4,0,0,0,0,0,0,0,0,0,0,0,0
4,Draw,1,5,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,0,18,12,0,0,0,0,0,0,0,0,0,0,0,0
268,0,18,13,0,0,0,0,0,0,0,0,0,0,0,0
269,0,18,14,0,0,0,0,0,0,0,0,0,0,0,0
270,0,18,15,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
from functions_load_and_transform import team_list

df = pd.read_csv("data/results.csv")

In [109]:
def score_counter(row, teams_dict):
    if not row["Winner"] == "0":
        teams_dict[row["Home Team"]][3] +=1
        teams_dict[row["Away Team"]][3] +=1
        if row["Winner"] == "Draw":
            teams_dict[row["Home Team"]][1] +=1
            teams_dict[row["Away Team"]][1] +=1
        else:
            # teams_dict[row["Winner"]][0] +=1
            if row["Home Team"] == row["Winner"]:
                # print("a")
                teams_dict[row["Home Team"]][0] +=1
            else:
                teams_dict[row["Away Team"]][2] +=1

def get_scoring(df):
    teams_dict = {}
    for team in team_list:
        teams_dict[team] = [0, 0, 0, 0] # Wins, Draws, Losses, Games Played
    df.apply(score_counter, teams_dict=teams_dict, axis=1)
    return teams_dict

teams_dict = get_scoring(pd.read_csv("data/results.csv"))
teams_dict

{'Kansas City Chiefs': [1, 0, 0, 1],
 'Philadelphia Eagles': [1, 0, 0, 1],
 'Atlanta Falcons': [1, 0, 0, 1],
 'Buffalo Bills': [0, 1, 0, 1],
 'Chicago Bears': [0, 1, 0, 1],
 'Cincinnati Bengals': [1, 0, 0, 1],
 'Indianapolis Colts': [1, 0, 0, 1],
 'Miami Dolphins': [0, 0, 0, 1],
 'New Orleans Saints': [0, 0, 0, 1],
 'New York Giants': [0, 0, 0, 1],
 'Los Angeles Chargers': [0, 0, 0, 0],
 'Seattle Seahawks': [0, 0, 0, 0],
 'Cleveland Browns': [1, 0, 0, 1],
 'Tampa Bay Buccaneers': [1, 0, 0, 1],
 'Detroit Lions': [1, 0, 0, 1],
 'San Francisco 49ers': [1, 0, 0, 1],
 'Dallas Cowboys': [0, 0, 0, 1],
 'Green Bay Packers': [0, 0, 0, 1],
 'Tennessee Titans': [0, 1, 0, 1],
 'Minnesota Vikings': [0, 0, 1, 1],
 'New England Patriots': [0, 0, 0, 1],
 'Washington Commanders': [0, 0, 0, 1],
 'Carolina Panthers': [0, 0, 1, 1],
 'Jacksonville Jaguars': [0, 0, 1, 1],
 'Baltimore Ravens': [0, 0, 0, 1],
 'Arizona Cardinals': [0, 1, 0, 1],
 'Denver Broncos': [0, 0, 0, 0],
 'Houston Texans': [0, 0, 0, 1],


In [114]:
pd.DataFrame.from_dict(teams_dict, orient="index", columns=["Wins", "Draws", "Losses", "Games Played"])

,Wins,Draws,Losses,Games Played
Kansas City Chiefs,1,0,0,1
Philadelphia Eagles,1,0,0,1
Atlanta Falcons,1,0,0,1
Buffalo Bills,0,1,0,1
Chicago Bears,0,1,0,1
Cincinnati Bengals,1,0,0,1
Indianapolis Colts,1,0,0,1
Miami Dolphins,0,0,0,1
New Orleans Saints,0,0,0,1
New York Giants,0,0,0,1


In [ ]:
# (W-T-L) Anzeige
# Daten in die Anzeige bringen